In [2]:
 import json
import collections
from PIL import Image
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 10  #default setting
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline
#Set all paths for data
train_path = "../data/Donnees_apprentissage/"
test_path = "../data/Donnees_test/"
resized_folder = "../data/resized_train"
resized_folder_ = "../data/resized_test"


#create a target forlder for resized pictures
if not os.path.exists(resized_folder):
    os.makedirs(resized_folder)
    
if not os.path.exists(resized_folder_):
    os.makedirs(resized_folder_)
#saving file:
meta = pd.read_csv("../data/label_learn.csv",sep = ";")

#purge target folder
target  = "../data/resized_test/"
filelist = [ f for f in os.listdir(target) if f.endswith(".jpg") ]
for f in filelist:
    os.remove(os.path.join(target, f))
def crop_square(img):
    min_side = min(img.size)/2
    half_the_width = img.size[0] / 2
    half_the_height = img.size[1] / 2
    img = img.crop(
        (
            half_the_width - min_side,
            half_the_height - min_side,
            half_the_width + min_side,
            half_the_height + min_side
        )
    )
    return img
def resize(source,target,size):
    dirs = os.listdir(source)
    dirs = [x for x in dirs if ".jpg" in x]
    for item in dirs:
        #print(source+"/"+item)
        im = Image.open(source+"/"+item)
        #f, e = os.path.splitext(source+"/"+item)
        im = pad_center(im)
        imResize = im.resize((size,size), Image.ANTIALIAS)
        imResize.save(target+item, 'JPEG', quality=100)
    return

dirs = os.listdir("../data/Donnees_apprentissage/")
dirs = [x for x in dirs if ".jpg" in x]
train_list = ["../data/Donnees_apprentissage/"+x for x in dirs if ".jpg" in x]

dirs = os.listdir("../data/Donnees_test/")
dirs = [x for x in dirs if ".jpg" in x]
test_list = ["../data/Donnees_test/"+x for x in dirs if ".jpg" in x]

In [6]:
%%time

import multiprocessing as mp
import datetime

size = 299
target = "../data/resized_train/"

def resize_(file_name):
    #print(source+"/"+item)
    im = Image.open(file_name)
    #f, e = os.path.splitext(file_name)
    im = crop_square(im)
    imResize = im.resize((size,size), Image.ANTIALIAS)
    item = file_name.split("/")[3]
    imResize.save(target+item, 'JPEG', quality=100)
    return 



try:
    pool = mp.Pool(processes=14)
    start_time=datetime.datetime.now()
    result = pool.map(resize_,train_list)
    pool.close()
    pool.join()
    elapsed=datetime.datetime.now()-start_time
    print("Done processing in %s" %(elapsed))
except Exception as e:
    print(e)
    pool.close()
    pool.join()
    print('Pool successfuly closed!')

Done processing in 0:04:05.575812
CPU times: user 216 ms, sys: 180 ms, total: 396 ms
Wall time: 4min 5s
